<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [12]:
import requests
from bs4 import BeautifulSoup
from selenium.webdriver import Firefox
import time

In [9]:
headers = {'Host': 'www.ecb.europa.eu',
'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:105.0) Gecko/20100101 Firefox/105.0',
'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8',
'Accept-Language': 'en-US,en;q=0.5',
'Accept-Encoding': 'gzip, deflate, br',
'Referer': 'https://www.ecb.europa.eu/press/govcdec/mopo/html/index.en.html',
'Connection': 'keep-alive',
'Upgrade-Insecure-Requests': '1',
'Sec-Fetch-Dest': 'document',
'Sec-Fetch-Mode': 'navigate',
'Sec-Fetch-Site': 'same-origin',
'Sec-Fetch-User': '?1',
'If-None-Match': "myra-39393ed9"}

In [285]:
# I found the page was dynamically loaded, so I used Selenium.

url = 'https://www.ecb.europa.eu/press/pressconf/html/index.en.html'
driver = Firefox()
driver.get(url)

In [302]:
target_links = []
link_dates = []
while True:
    old_len = len(target_links)
    links = driver.find_elements(by='tag name', value='dd')
    dates = driver.find_elements(by='tag name', value='dt')
    
    for date, link in zip(dates, links):
        link_text = link.text.lower()
        link_wanted = ('introductory statement' in link_text) or ('monetary policy statement' in link_text)
        if link not in target_links and link_wanted:
            target_links.append(link)
            link_dates.append(date.text.strip())
    if len(target_links) == old_len:
        break
    driver.execute_script("arguments[0].scrollIntoView(true);", link)
    time.sleep(5)

In [307]:
urls = []
for link in target_links:
    a_href = link.find_element(by='tag name', value='a')
    urls.append(a_href.get_attribute('href'))

In [308]:
import re
DATE_PATTERN = re.compile('(\d{1,2} (January|February|March|April|May|June|July|August|September|October|November|December) \d{4})')

In [294]:
from tqdm.notebook import tqdm

data = []
for i, url in enumerate(tqdm(urls)):
    page = requests.get(url).content
    soup = BeautifulSoup(page, parser='lxml-parser', features='html')
    textbox = soup.find('main')
    
    name = soup.find('div', class_='title').get_text().strip()
    
    title_text = textbox.find('h2', class_='ecb-pressContentSubtitle').get_text()
    
    title_text = title_text.replace('-', ' ')
    title_text = title_text.replace('European Central Bank', 'ECB')
    title_text = title_text.replace(' and ', ', and ')
    
    ps = textbox.find_all('p')

    items = title_text.split(',')

    president = None
    vice_president = None
    location = None
    date = None

    #Check for president
    Pres = [x for x in items if 'President' in x and 'ECB' in x and 'Vice' not in x]
    if len(Pres) == 1:
        p_idx = items.index(Pres[0]) - 1
        president = items[p_idx].strip()

    # Check for Vice-President
    VP = [x for x in items if 'Vice President' in x and 'ECB' in x]
    if len(VP) == 1:
        vp_idx =  items.index(VP[0]) - 1 
        vice_president = items[vp_idx]
        
    # Check for a location
    # Case 1: items has 6 elements, then location is number 5
    if len(items) == 6:
        location = items[4]
    # Case 2: There was no comma separating the VP and the location.
    elif len(VP) > 0:
        candidate = VP[0].replace('Vice President of the ECB', '' ).strip()
        if candidate != '':
            location = candidate
            
    info = dict(
        date = date,
        location = location,
        president = president,
        vice_president = vice_president,
        title = name,
        text = '\n'.join([x.get_text() for x in ps]),
        url = url
    )

    data.append(info)

  0%|          | 0/259 [00:00<?, ?it/s]

In [373]:
df = pd.DataFrame(data)
df.date = pd.Series(link_dates)
df = df[~df.title.isin(df.title.value_counts()[df.title.value_counts() == 1].index)]

In [374]:
df.to_csv('ECB_introductory_statements_raw.csv')